# Chapter 3: Core AWS AI Services - Executable Examples

This chapter provides executable examples for the core AWS AI services. Each example is self-contained and can be run with minimal setup.

## Prerequisites

- An AWS account with appropriate permissions
- AWS CLI configured with your credentials
- Python 3.7 or later
- Required Python packages: boto3, pandas, numpy, scikit-learn, matplotlib

Install required packages:

In [ ]:
%%bash
pip install boto3 pandas numpy scikit-learn matplotlib

## 1. Amazon SageMaker: Iris Flower Classification

This example uses the Iris dataset to demonstrate Amazon SageMaker's capabilities.

In [ ]:
import boto3
import sagemaker
import pandas as pd
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sagemaker import get_execution_role
from sagemaker.amazon.amazon_estimator import get_image_uri

# Load and prepare data
iris = load_iris()
X, y = iris.data, iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

train_data = np.column_stack((X_train, y_train))
test_data = np.column_stack((X_test, y_test))

pd.DataFrame(train_data).to_csv('train.csv', header=False, index=False)
pd.DataFrame(test_data).to_csv('test.csv', header=False, index=False)

# Set up SageMaker session
sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()
prefix = 'sagemaker/DEMO-iris'

# Upload data to S3
train_location = sagemaker_session.upload_data('train.csv', bucket=bucket, key_prefix=prefix)
test_location = sagemaker_session.upload_data('test.csv', bucket=bucket, key_prefix=prefix)

# Set up estimator
container = get_image_uri(boto3.Session().region_name, 'xgboost')
role = get_execution_role()

xgb = sagemaker.estimator.Estimator(container,
                                    role, 
                                    train_instance_count=1, 
                                    train_instance_type='ml.m4.xlarge',
                                    output_path=f's3://{bucket}/{prefix}/output',
                                    sagemaker_session=sagemaker_session)

xgb.set_hyperparameters(max_depth=5,
                        eta=0.2,
                        gamma=4,
                        min_child_weight=6,
                        subsample=0.8,
                        objective='multi:softmax',
                        num_class=3,
                        num_round=100)

# Train the model
xgb.fit({'train': train_location, 'validation': test_location})

# Deploy the model
predictor = xgb.deploy(initial_instance_count=1, instance_type='ml.t2.medium')

# Make predictions
test_data_np = np.loadtxt('test.csv', delimiter=',')
test_features = test_data_np[:, :-1]

predictions = predictor.predict(test_features).decode('utf-8')
predictions = np.fromstring(predictions[1:-1], sep=',')

# Evaluate the model
from sklearn.metrics import accuracy_score, classification_report
accuracy = accuracy_score(y_test, predictions)
print(f"Model Accuracy: {accuracy}")
print("Classification Report:")
print(classification_report(y_test, predictions, target_names=iris.target_names))

# Clean up
predictor.delete_endpoint()

## 2. Amazon Comprehend: Sentiment Analysis

This example uses Amazon Comprehend to perform sentiment analysis on a sample text.

In [ ]:
import boto3
import json

comprehend = boto3.client('comprehend')

text = "I love using AWS services. They make machine learning so much easier!"

# Detect sentiment
sentiment = comprehend.detect_sentiment(Text=text, LanguageCode='en')
print(f"Sentiment: {sentiment['Sentiment']}")
print(f"Sentiment Scores: {json.dumps(sentiment['SentimentScore'], indent=2)}")

# Detect entities
entities = comprehend.detect_entities(Text=text, LanguageCode='en')
print("\nEntities:")
for entity in entities['Entities']:
    print(f"  {entity['Text']} ({entity['Type']})")

# Detect key phrases
phrases = comprehend.detect_key_phrases(Text=text, LanguageCode='en')
print("\nKey Phrases:")
for phrase in phrases['KeyPhrases']:
    print(f"  {phrase['Text']}")

# Detect language
languages = comprehend.detect_dominant_language(Text=text)
print("\nDominant Language:")
for language in languages['Languages']:
    print(f"  {language['LanguageCode']} ({language['Score']})")

## 3. Amazon Rekognition: Object and Face Detection

This example uses Amazon Rekognition to detect objects and faces in an image.

In [ ]:
import boto3
import requests
from PIL import Image, ImageDraw, ImageFont

# Download a sample image
image_url = "https://images.pexels.com/photos/1516680/pexels-photo-1516680.jpeg"
response = requests.get(image_url)
with open("sample_image.jpg", "wb") as f:
    f.write(response.content)

# Set up Rekognition client
rekognition = boto3.client('rekognition')

# Detect objects
with open("sample_image.jpg", "rb") as image_file:
    response = rekognition.detect_labels(Image={'Bytes': image_file.read()})

print("Detected objects:")
for label in response['Labels']:
    print(f"  {label['Name']} (Confidence: {label['Confidence']:.2f})")

# Detect faces
with open("sample_image.jpg", "rb") as image_file:
    response = rekognition.detect_faces(Image={'Bytes': image_file.read()}, Attributes=['ALL'])

print("\nDetected faces:")
for face in response['FaceDetails']:
    print(f"  Age range: {face['AgeRange']['Low']}-{face['AgeRange']['High']}")
    print(f"  Gender: {face['Gender']['Value']} (Confidence: {face['Gender']['Confidence']:.2f})")
    print(f"  Emotions: {face['Emotions'][0]['Type']} (Confidence: {face['Emotions'][0]['Confidence']:.2f})")

# Draw bounding boxes
image = Image.open("sample_image.jpg")
draw = ImageDraw.Draw(image)

for face in response['FaceDetails']:
    box = face['BoundingBox']
    left = image.width * box['Left']
    top = image.height * box['Top']
    width = image.width * box['Width']
    height = image.height * box['Height']
    draw.rectangle([left, top, left + width, top + height], outline='red', width=2)

image.save("output_image.jpg")
print("\nImage with bounding boxes saved as 'output_image.jpg'")

## 4. Amazon Forecast: Time Series Forecasting

This example demonstrates how to use Amazon Forecast for time series prediction. Note that this is a simplified example and actual forecast creation takes more time and data.

In [ ]:
import boto3
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# Create sample time series data
start_date = datetime(2022, 1, 1)
dates = [start_date + timedelta(days=i) for i in range(365)]
values = np.sin(np.arange(365) * 2 * np.pi / 365) * 50 + 100 + np.random.normal(0, 10, 365)
df = pd.DataFrame({'date': dates, 'value': values})
df['date'] = df['date'].dt.strftime('%Y-%m-%d')
df.to_csv('time_series_data.csv', index=False)

# Set up Forecast client
forecast = boto3.client('forecast')
s3 = boto3.client('s3')

# Upload data to S3
bucket_name = 'your-bucket-name'  # Replace with your S3 bucket name
s3.upload_file('time_series_data.csv', bucket_name, 'forecast_data/time_series_data.csv')

# Create dataset group
response = forecast.create_dataset_group(
    DatasetGroupName='sample-forecast-dataset-group',
    Domain='CUSTOM',
    DatasetArns=[]
)
dataset_group_arn = response['DatasetGroupArn']

# Create dataset
response = forecast.create_dataset(
    DatasetName='sample-forecast-dataset',
    Domain='CUSTOM',
    DatasetType='TARGET_TIME_SERIES',
    DataFrequency='D',
    Schema={
        'Attributes': [
            {'AttributeName': 'date', 'AttributeType': 'timestamp'},
            {'AttributeName': 'value', 'AttributeType': 'float'},
            {'AttributeName': 'item_id', 'AttributeType': 'string'}
        ]
    }
)
dataset_arn = response['DatasetArn']

# Create dataset import job
response = forecast.create_dataset_import_job(
    DatasetImportJobName='sample-import-job',
    DatasetArn=dataset_arn,
    DataSource={
        'S3Config': {
            'Path': f's3://{bucket_name}/forecast_data/time_series_data.csv',
            'RoleArn': 'your-role-arn'  # Replace with your IAM role ARN
        }
    },
    TimestampFormat='yyyy-MM-dd'
)

# Wait for import job to complete (this may take several minutes)
import time
while True:
    response = forecast.describe_dataset_import_job(
        DatasetImportJobArn=response['DatasetImportJobArn']
    )
    if response['Status'] in ['ACTIVE', 'CREATE_FAILED']:
        break
    time.sleep(30)

print("Dataset import job status:", response['Status'])

# Note: In a real scenario, you would proceed to create a predictor and forecast
# However, these steps can take significant time and resources
print("To create a predictor and forecast, use the following steps:")
print("1. forecast.create_predictor()")
print("2. forecast.create_forecast()")
print("3. forecast.query_forecast()")

## 5. Amazon Personalize: Recommendation System

This example demonstrates how to set up and use Amazon Personalize for a simple recommendation system. Note that this is a simplified example and actual model training takes more time and data.

In [ ]:
import boto3
import pandas as pd
import json
from time import sleep

# Create sample interaction data
interactions = pd.DataFrame({
    'USER_ID': ['user1', 'user1', 'user2', 'user2', 'user3'],
    'ITEM_ID': ['item1', 'item2', 'item2', 'item3', 'item1'],
    'TIMESTAMP': [1625097600, 1625184000, 1625270400, 1625356800, 1625443200]
})
interactions.to_csv('interactions.csv', index=False)

# Set up Personalize client
personalize = boto3.client('personalize')
s3 = boto3.client('s3')

# Upload data to S3
bucket_name = 'your-bucket-name'  # Replace with your S3 bucket name
s3.upload_file('interactions.csv', bucket_name, 'personalize_data/interactions.csv')

# Create dataset group
response = personalize.create_dataset_group(name='sample-dataset-group')
dataset_group_arn = response['datasetGroupArn']

# Create schema
schema = {
    "type": "record",
    "name": "Interactions",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {"name": "USER_ID", "type": "string"},
        {"name": "ITEM_ID", "type": "string"},
        {"name": "TIMESTAMP", "type": "long"}
    ],
    "version": "1.0"
}

create_schema_response = personalize.create_schema(
    name='sample-interactions-schema',
    schema=json.dumps(schema)
)
schema_arn = create_schema_response['schemaArn']

# Create dataset
create_dataset_response = personalize.create_dataset(
    name='sample-interactions-dataset',
    schemaArn=schema_arn,
    datasetGroupArn=dataset_group_arn,
    datasetType='Interactions'
)
dataset_arn = create_dataset_response['datasetArn']

# Create dataset import job
create_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName='sample-import-job',
    datasetArn=dataset_arn,
    dataSource={'dataLocation': f's3://{bucket_name}/personalize_data/interactions.csv'},
    roleArn='your-role-arn'  # Replace with your IAM role ARN
)

# Wait for import job to complete (this may take several minutes)
import_job_arn = create_dataset_import_job_response['datasetImportJobArn']
while True:
    describe_dataset_import_job_response = personalize.describe_dataset_import_job(
        datasetImportJobArn=import_job_arn
    )
    status = describe_dataset_import_job_response['datasetImportJob']['status']
    print(f"Import job status: {status}")
    if status in ['ACTIVE', 'CREATE FAILED']:
        break
    sleep(30)

print("Dataset import job completed.")

# Note: In a real scenario, you would proceed to create a solution, campaign, and get recommendations
# However, these steps can take significant time and resources
print("To create a solution, campaign, and get recommendations, use the following steps:")
print("1. personalize.create_solution()")
print("2. personalize.create_solution_version()")
print("3. personalize.create_campaign()")
print("4. personalize_runtime.get_recommendations()")

## 6. Amazon Polly: Text-to-Speech

Amazon Polly is a service that turns text into lifelike speech. This example demonstrates how to use Polly to synthesize speech from text.

In [ ]:
import boto3
import os
from contextlib import closing

polly_client = boto3.client('polly')

text = "Hello, this is a test of Amazon Polly. Isn't text-to-speech amazing?"

response = polly_client.synthesize_speech(
    Text=text,
    OutputFormat='mp3',
    VoiceId='Joanna'
)

# Save the audio stream to a file
if "AudioStream" in response:
    with closing(response["AudioStream"]) as stream:
        output = os.path.join(".", "speech.mp3")
        with open(output, "wb") as file:
            file.write(stream.read())
    print(f"Speech saved to {output}")

# To play the audio, you can use a library like playsound
# pip install playsound
try:
    from playsound import playsound
    playsound('speech.mp3')
except ImportError:
    print("To play the audio, install playsound: pip install playsound")

# List available voices
voices = polly_client.describe_voices()
print("\nAvailable voices:")
for voice in voices['Voices'][:5]:  # Printing first 5 for brevity
    print(f"Name: {voice['Name']}, Gender: {voice['Gender']}, Language: {voice['LanguageCode']}")

## 7. Amazon Transcribe: Speech-to-Text

Amazon Transcribe is an automatic speech recognition service. This example shows how to transcribe an audio file.

In [ ]:
import boto3
import time
import urllib.request

s3_client = boto3.client('s3')
transcribe_client = boto3.client('transcribe')

# Download a sample audio file
audio_url = "https://github.com/OpenMindClub/omnivore/raw/main/audios/turing_test_audio.mp3"
urllib.request.urlretrieve(audio_url, "sample_audio.mp3")

# Upload the audio file to S3
bucket_name = 'your-bucket-name'  # Replace with your S3 bucket name
s3_client.upload_file('sample_audio.mp3', bucket_name, 'sample_audio.mp3')

# Start transcription job
job_name = 'sample-transcription-job'
job_uri = f's3://{bucket_name}/sample_audio.mp3'

transcribe_client.start_transcription_job(
    TranscriptionJobName=job_name,
    Media={'MediaFileUri': job_uri},
    MediaFormat='mp3',
    LanguageCode='en-US'
)

# Wait for the job to complete
while True:
    status = transcribe_client.get_transcription_job(TranscriptionJobName=job_name)
    if status['TranscriptionJob']['TranscriptionJobStatus'] in ['COMPLETED', 'FAILED']:
        break
    print("Not ready yet...")
    time.sleep(5)

if status['TranscriptionJob']['TranscriptionJobStatus'] == 'COMPLETED':
    response = urllib.request.urlopen(status['TranscriptionJob']['Transcript']['TranscriptFileUri'])
    data = response.read()
    text = data.decode('utf-8')
    print("\nTranscription:")
    print(text[:500] + "...")  # Printing first 500 characters for brevity
else:
    print("Transcription failed.")

# Clean up
transcribe_client.delete_transcription_job(TranscriptionJobName=job_name)
s3_client.delete_object(Bucket=bucket_name, Key='sample_audio.mp3')

## 8. Amazon Translate: Language Translation

Amazon Translate is a neural machine translation service. This example demonstrates how to translate text between languages.

In [ ]:
import boto3

translate_client = boto3.client('translate')

text = "Hello, world. How are you doing today?"

# Translate to Spanish
response = translate_client.translate_text(
    Text=text,
    SourceLanguageCode='en',
    TargetLanguageCode='es'
)

print(f"Original text: {text}")
print(f"Translated text: {response['TranslatedText']}")

# Detect language
response = translate_client.translate_text(
    Text=text,
    SourceLanguageCode='auto',
    TargetLanguageCode='de'
)

print(f"\nDetected source language: {response['SourceLanguageCode']}")
print(f"Translated to German: {response['TranslatedText']}")

# List supported languages
response = translate_client.list_languages()
print("\nSupported languages:")
for language in response['Languages'][:10]:  # Printing first 10 for brevity
    print(f"Language Code: {language['LanguageCode']}, Name: {language['LanguageName']}")

## Conclusion
These examples demonstrate how to use the core AWS AI services:

1. Amazon SageMaker for machine learning model training and deployment
2. Amazon Comprehend for natural language processing
3. Amazon Rekognition for image and video analysis
4. Amazon Forecast for time series forecasting
5. Amazon Personalize for building recommendation systems
6. Amazon Polly for text-to-speech conversion
7. Amazon Transcribe for speech-to-text conversion
8. Amazon Translate for language translation


Remember:
- Always clean up your resources to avoid unnecessary charges.
- These examples use small datasets for demonstration. Real-world applications typically involve larger datasets and more complex workflows.
- Ensure you have the necessary AWS permissions to run these examples.
- Some services (like Forecast and Personalize) may take significant time to process data and train models in real-world scenarios.

As you become more familiar with these services, explore their advanced features and integrate them into your AI/ML projects.